In [1]:
import requests, shutil, time, cv2
from bs4 import BeautifulSoup
import numpy as np
from keras.models import load_model
from preprocessBatch import preprocessing

allowedChars = 'ACDEFGHJKLNPQRTUVXYZ2346789';
CAPTCHA_IMG = "captcha.jpg"
PROCESSED_IMG = "preprocessing.jpg"
FOLDER = "captcha/"
CSV_FILE = "auto-train.csv"

Using TensorFlow backend.


start to process image from index: 2000
completed


In [2]:
print('model loading...')
model = load_model("twse_cnn_model.hdf5")
print('loading completed')

W0917 22:41:35.665929 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 22:41:35.689950 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0917 22:41:35.707770 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 22:41:35.710916 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0917 22:41:35.725850 4385134016 deprecati

model loading...


W0917 22:41:35.894305 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0917 22:41:36.006790 4385134016 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0917 22:41:36.360463 4385134016 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


loading completed


In [3]:
i = 850

while i < 2000:
    resp = requests.get("https://bsr.twse.com.tw/bshtm/bsMenu.aspx")
    soup = BeautifulSoup(resp.text, 'html.parser')
    img_tags = soup.select("#Panel_bshtm img")
    src = img_tags[0].get('src')

    resp = requests.get("https://bsr.twse.com.tw/bshtm/" + src, stream=True)
    if resp.status_code == 200:
        with open(CAPTCHA_IMG, 'wb') as f:
            resp.raw.decode_content = True
            shutil.copyfileobj(resp.raw, f)
            
    preprocessing(CAPTCHA_IMG, PROCESSED_IMG)
    train_data = np.stack([np.array(cv2.imread(PROCESSED_IMG))/255.0])
    prediction = model.predict(train_data)

    predict_captcha = ''
    for predict in prediction:
        value = np.argmax(predict[0])
        predict_captcha += allowedChars[value]

    print("predict_captcha: " + predict_captcha)
    
    payload = {}
    acceptable_input = ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION', 'RadioButton_Normal',
                        'TextBox_Stkno', 'CaptchaControl1', 'btnOK']

    inputs = soup.select("input")
    for elem in inputs:
        if elem.get("name") in acceptable_input:
            if elem.get("value") != None:
                payload[elem.get("name")] = elem.get("value")
            else:
                payload[elem.get("name")] = ""
                
    payload['TextBox_Stkno'] = '2330'
    payload['CaptchaControl1'] = predict_captcha
    
    resp = requests.post("https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload)
    if '驗證碼錯誤!' in resp.text:
        print('驗證碼錯誤')
    elif '驗證碼已逾期!' in resp.text:
        print('驗證碼已逾期')
    elif 'HyperLink_DownloadCSV' in resp.text:
        # add to label\n",
        i += 1
        print("success, add " + str(i) + ".jpg with captcha: " + predict_captcha)
        shutil.copyfile(CAPTCHA_IMG, FOLDER + str(i) + ".jpg")
        with open(CSV_FILE,'a') as fd:
            fd.write(predict_captcha + "\n")
            
    time.sleep(1)


predict_captcha: RFTXP
驗證碼錯誤
predict_captcha: ZX29N
驗證碼錯誤
predict_captcha: X2HAV
驗證碼錯誤
predict_captcha: G8N33
驗證碼錯誤
predict_captcha: 4EE9A
驗證碼錯誤
predict_captcha: DYERD
驗證碼錯誤
predict_captcha: TX7AL
success, add 851.jpg with captcha: TX7AL
predict_captcha: L3Q4U
驗證碼錯誤
predict_captcha: 2G78R
驗證碼錯誤
predict_captcha: XECEA
success, add 852.jpg with captcha: XECEA
predict_captcha: 4HTXR
驗證碼錯誤
predict_captcha: 2JUER
驗證碼錯誤
predict_captcha: ZDAP8
success, add 853.jpg with captcha: ZDAP8
predict_captcha: TALYN
驗證碼錯誤
predict_captcha: XP4XV
驗證碼錯誤
predict_captcha: UQXG4
success, add 854.jpg with captcha: UQXG4
predict_captcha: PT98C
驗證碼錯誤
predict_captcha: C3JEJ
success, add 855.jpg with captcha: C3JEJ
predict_captcha: CYEZ2
驗證碼錯誤
predict_captcha: HG9FA
驗證碼錯誤
predict_captcha: CUJ8A
驗證碼錯誤
predict_captcha: HNT7R
驗證碼錯誤
predict_captcha: 2QNH3
驗證碼錯誤
predict_captcha: JNALJ
success, add 856.jpg with captcha: JNALJ
predict_captcha: ZF37R
驗證碼錯誤
predict_captcha: 32QGT
驗證碼錯誤
predict_captcha: F2C89
驗證碼錯誤
predict

predict_captcha: H9TNK
驗證碼錯誤
predict_captcha: FP3GX
驗證碼錯誤
predict_captcha: Q8TD2
驗證碼錯誤
predict_captcha: C3AJ2
驗證碼錯誤
predict_captcha: VG3V9
驗證碼錯誤
predict_captcha: J3EAY
success, add 888.jpg with captcha: J3EAY
predict_captcha: PRHEZ
驗證碼錯誤
predict_captcha: JZTC3
驗證碼錯誤
predict_captcha: CFTAN
驗證碼錯誤
predict_captcha: P3FEH
驗證碼錯誤
predict_captcha: 9ZHXX
驗證碼錯誤
predict_captcha: 8HJPX
驗證碼錯誤
predict_captcha: RJD86
驗證碼錯誤
predict_captcha: C7TZX
success, add 889.jpg with captcha: C7TZX
predict_captcha: CHP4X
驗證碼錯誤
predict_captcha: CJ2N8
驗證碼錯誤
predict_captcha: 4QDXE
驗證碼錯誤
predict_captcha: T3TA7
驗證碼錯誤
predict_captcha: 33Y9P
驗證碼錯誤
predict_captcha: PTFEH
驗證碼錯誤
predict_captcha: T8P8J
驗證碼錯誤
predict_captcha: 4JA8D
驗證碼錯誤
predict_captcha: X9ANR
驗證碼錯誤
predict_captcha: UZEX8
驗證碼錯誤
predict_captcha: A39FT
驗證碼錯誤
predict_captcha: A3PL3
驗證碼錯誤
predict_captcha: CJH78
驗證碼錯誤
predict_captcha: FHJA9
驗證碼錯誤
predict_captcha: QYF39
success, add 890.jpg with captcha: QYF39
predict_captcha: CUQEQ
驗證碼錯誤
predict_captcha: 3877N
驗證

predict_captcha: 2HNCL
驗證碼錯誤
predict_captcha: 2QC36
驗證碼錯誤
predict_captcha: CCDVH
驗證碼錯誤
predict_captcha: H9N3V
驗證碼錯誤
predict_captcha: U7Z8E
驗證碼錯誤
predict_captcha: RV7H8
驗證碼錯誤
predict_captcha: U82VA
驗證碼錯誤
predict_captcha: T4Q4C
驗證碼錯誤
predict_captcha: 4LEGL
驗證碼錯誤
predict_captcha: 8X7QU
驗證碼錯誤
predict_captcha: 9DJEG
驗證碼錯誤
predict_captcha: GEZEV
驗證碼錯誤
predict_captcha: 7FPVD
驗證碼錯誤
predict_captcha: H2AHG
驗證碼錯誤
predict_captcha: 233Z3
驗證碼錯誤
predict_captcha: JHUG8
驗證碼錯誤
predict_captcha: U8VGY
驗證碼錯誤
predict_captcha: 2HNXD
驗證碼錯誤
predict_captcha: HNXTA
success, add 915.jpg with captcha: HNXTA
predict_captcha: D2V8T
驗證碼錯誤
predict_captcha: RHKHP
驗證碼錯誤
predict_captcha: QLKK9
驗證碼錯誤
predict_captcha: Y8E36
驗證碼錯誤
predict_captcha: 2EPXQ
驗證碼錯誤
predict_captcha: ZPJFE
驗證碼錯誤
predict_captcha: VT9GV
success, add 916.jpg with captcha: VT9GV
predict_captcha: TL4A4
驗證碼錯誤
predict_captcha: REYZ8
驗證碼錯誤
predict_captcha: 4278V
驗證碼錯誤
predict_captcha: A3PUJ
驗證碼錯誤
predict_captcha: UF7UR
驗證碼錯誤
predict_captcha: GQ68V
驗證碼錯誤
pr

predict_captcha: HKTLE
success, add 944.jpg with captcha: HKTLE
predict_captcha: HRZY3
success, add 945.jpg with captcha: HRZY3
predict_captcha: R8TPE
驗證碼錯誤
predict_captcha: FTJ2H
驗證碼錯誤
predict_captcha: DP3GL
驗證碼錯誤
predict_captcha: Y4QLC
驗證碼錯誤
predict_captcha: AZNPA
驗證碼錯誤
predict_captcha: 8KAPE
驗證碼錯誤
predict_captcha: LGLDZ
success, add 946.jpg with captcha: LGLDZ
predict_captcha: CZK44
success, add 947.jpg with captcha: CZK44
predict_captcha: 4448V
驗證碼錯誤
predict_captcha: Z3QPZ
驗證碼錯誤
predict_captcha: D33Z8
驗證碼錯誤
predict_captcha: 432QN
驗證碼錯誤
predict_captcha: CQ7VZ
驗證碼錯誤
predict_captcha: 9UF74
驗證碼錯誤
predict_captcha: FN8NZ
驗證碼錯誤
predict_captcha: CQE4R
驗證碼錯誤
predict_captcha: AX2PV
驗證碼錯誤
predict_captcha: D97L8
驗證碼錯誤
predict_captcha: A8HZR
驗證碼錯誤
predict_captcha: UJNQ4
驗證碼錯誤
predict_captcha: PCJVD
驗證碼錯誤
predict_captcha: VLYNT
驗證碼錯誤
predict_captcha: ZT4T8
驗證碼錯誤
predict_captcha: FPEDP
success, add 948.jpg with captcha: FPEDP
predict_captcha: CYJH2
驗證碼錯誤
predict_captcha: 49JPK
驗證碼錯誤
predict_captc

predict_captcha: TYDK4
驗證碼錯誤
predict_captcha: 7VQRJ
驗證碼錯誤
predict_captcha: DTTE2
驗證碼錯誤
predict_captcha: 2GGD3
驗證碼錯誤
predict_captcha: NFZTN
驗證碼錯誤
predict_captcha: KEFDZ
驗證碼錯誤
predict_captcha: G8QGU
success, add 986.jpg with captcha: G8QGU
predict_captcha: UQKGR
驗證碼錯誤
predict_captcha: 836HD
驗證碼錯誤
predict_captcha: YFRHL
success, add 987.jpg with captcha: YFRHL
predict_captcha: 3Y642
驗證碼錯誤
predict_captcha: ZJURC
驗證碼錯誤
predict_captcha: HR7TT
驗證碼錯誤
predict_captcha: HQD8G
驗證碼錯誤
predict_captcha: HA3LK
驗證碼錯誤
predict_captcha: 4LQ26
驗證碼錯誤
predict_captcha: HEPVT
驗證碼錯誤
predict_captcha: TXQ2P
驗證碼錯誤
predict_captcha: C2LDD
驗證碼錯誤
predict_captcha: 2GT38
predict_captcha: C3AP2
驗證碼錯誤
predict_captcha: URG2R
success, add 988.jpg with captcha: URG2R
predict_captcha: A8TF8
驗證碼錯誤
predict_captcha: PZA2H
驗證碼錯誤
predict_captcha: R8E8Z
驗證碼錯誤
predict_captcha: ZN8XC
驗證碼錯誤
predict_captcha: VLPGN
success, add 989.jpg with captcha: VLPGN
predict_captcha: 6XH92
驗證碼錯誤
predict_captcha: QUKLE
驗證碼錯誤
predict_captcha: EUKU7
驗證

...

success, add 1173.jpg with captcha: Q98JL
predict_captcha: TJ4N7
驗證碼錯誤
predict_captcha: HQQVF
驗證碼錯誤
predict_captcha: CZLYT
驗證碼錯誤
predict_captcha: Q2LZ9
success, add 1174.jpg with captcha: Q2LZ9
predict_captcha: YXEEK
驗證碼錯誤
predict_captcha: XLHXT
驗證碼錯誤
predict_captcha: RJU86
驗證碼錯誤
predict_captcha: TDHL3
驗證碼錯誤
predict_captcha: 37PUV
驗證碼錯誤
predict_captcha: 8REQA
驗證碼錯誤
predict_captcha: CZQP8
驗證碼錯誤
predict_captcha: CAFY4
驗證碼錯誤
predict_captcha: XHKKC
驗證碼錯誤
predict_captcha: KUHZJ
驗證碼錯誤
predict_captcha: P9RLP
驗證碼錯誤
predict_captcha: ADF8V
驗證碼錯誤
predict_captcha: C7N9C
驗證碼錯誤
predict_captcha: HT4FR
驗證碼錯誤
predict_captcha: 28496
驗證碼錯誤
predict_captcha: JTU84
驗證碼錯誤
predict_captcha: CYYDJ
驗證碼錯誤
predict_captcha: KXC4R
success, add 1175.jpg with captcha: KXC4R
predict_captcha: RLHQ8
驗證碼錯誤
predict_captcha: 9ELP8
success, add 1176.jpg with captcha: 9ELP8
predict_captcha: 6PGP6
驗證碼錯誤
predict_captcha: ZXGGG
驗證碼錯誤
predict_captcha: QLNG2
驗證碼錯誤
predict_captcha: CYQ49
驗證碼錯誤
predict_captcha: C326P
驗證碼錯誤
predict_c

predict_captcha: 8L6DU
驗證碼錯誤
predict_captcha: TQKR8
驗證碼錯誤
predict_captcha: U8Q7A
驗證碼錯誤
predict_captcha: KHXUY
驗證碼錯誤
predict_captcha: KZ47T
驗證碼錯誤
predict_captcha: 8TQXA
驗證碼錯誤
predict_captcha: GJGPC
驗證碼錯誤
predict_captcha: H4NV7
驗證碼錯誤
predict_captcha: HDTNP
驗證碼錯誤
predict_captcha: GJYFN
驗證碼錯誤
predict_captcha: HGAL8
驗證碼錯誤
predict_captcha: CPFFC
驗證碼錯誤
predict_captcha: JLPLH
驗證碼錯誤
predict_captcha: U4TVN
success, add 1206.jpg with captcha: U4TVN
predict_captcha: HDYNV
驗證碼錯誤
predict_captcha: HK4P8
驗證碼錯誤
predict_captcha: Z7TQZ
驗證碼錯誤
predict_captcha: HAAG8
驗證碼錯誤
predict_captcha: F46GV
驗證碼錯誤
predict_captcha: EE8V6
驗證碼錯誤
predict_captcha: YGXQN
驗證碼錯誤
predict_captcha: 3YAYJ
驗證碼錯誤
predict_captcha: HGEUE
驗證碼錯誤
predict_captcha: EP9JN
驗證碼錯誤
predict_captcha: 2XAEV
驗證碼錯誤
predict_captcha: NE2D7
驗證碼錯誤
predict_captcha: NCU46
驗證碼錯誤
predict_captcha: T8DK3
success, add 1207.jpg with captcha: T8DK3
predict_captcha: XLQ3Y
驗證碼錯誤
predict_captcha: 4J3EE
success, add 1208.jpg with captcha: 4J3EE
predict_captcha: CK28C

驗證碼錯誤
predict_captcha: 986GJ
驗證碼錯誤
predict_captcha: HH7FV
驗證碼錯誤
predict_captcha: QT98R
驗證碼錯誤
predict_captcha: JDA72
驗證碼錯誤
predict_captcha: ZERKC
驗證碼錯誤
predict_captcha: UQRRY
驗證碼錯誤
predict_captcha: CUFJF
驗證碼錯誤
predict_captcha: Q2KPR
驗證碼錯誤
predict_captcha: 4ZJXD
驗證碼錯誤
predict_captcha: THHLF
success, add 1235.jpg with captcha: THHLF
predict_captcha: YQA6D
驗證碼錯誤
predict_captcha: K98ZG
驗證碼錯誤
predict_captcha: XH3HF
驗證碼錯誤
predict_captcha: YQ8P8
驗證碼錯誤
predict_captcha: 2XD4C
驗證碼錯誤
predict_captcha: UZ2V2
success, add 1236.jpg with captcha: UZ2V2
predict_captcha: 9JDEF
驗證碼錯誤
predict_captcha: CGFVY
驗證碼錯誤
predict_captcha: XN4T7
success, add 1237.jpg with captcha: XN4T7
predict_captcha: KART7
success, add 1238.jpg with captcha: KART7
predict_captcha: PP9F6
驗證碼錯誤
predict_captcha: Y3F73
驗證碼錯誤
predict_captcha: TDKYC
success, add 1239.jpg with captcha: TDKYC
predict_captcha: 4UL84
驗證碼錯誤
predict_captcha: 8378A
驗證碼錯誤
predict_captcha: 7ZEEA
驗證碼錯誤
predict_captcha: YYZVV
驗證碼錯誤
predict_captcha: UN3Z7
驗證碼錯誤
pr

驗證碼錯誤
predict_captcha: PLVFG
驗證碼錯誤
predict_captcha: KXKQN
success, add 1266.jpg with captcha: KXKQN
predict_captcha: L9F8Y
驗證碼錯誤
predict_captcha: 7QXQJ
驗證碼錯誤
predict_captcha: 4NRK6
驗證碼錯誤
predict_captcha: 2QLFV
驗證碼錯誤
predict_captcha: 8RH9A
驗證碼錯誤
predict_captcha: HG7PY
驗證碼錯誤
predict_captcha: TQE99
驗證碼錯誤
predict_captcha: CY9NN
驗證碼錯誤
predict_captcha: YJRJE
驗證碼錯誤
predict_captcha: 2GA23
驗證碼錯誤
predict_captcha: 9FQPE
驗證碼錯誤
predict_captcha: V8YQP
驗證碼錯誤
predict_captcha: 6L7ZK
success, add 1267.jpg with captcha: 6L7ZK
predict_captcha: FNAUD
驗證碼錯誤
predict_captcha: DQ9FV
驗證碼錯誤
predict_captcha: 4QPHH
驗證碼錯誤
predict_captcha: HV3Q8
驗證碼錯誤
predict_captcha: 6QQKY
驗證碼錯誤
predict_captcha: FYRUU
success, add 1268.jpg with captcha: FYRUU
predict_captcha: 3HGLX
驗證碼錯誤
predict_captcha: QENVX
success, add 1269.jpg with captcha: QENVX
predict_captcha: 8TQNR
驗證碼錯誤
predict_captcha: ADGXT
驗證碼錯誤
predict_captcha: 2LRRG
success, add 1270.jpg with captcha: 2LRRG
predict_captcha: CNVL9
success, add 1271.jpg with captcha: C

KeyboardInterrupt: 